# camel case by index

Instead of using a dictionary to determine word boundaries, we can use index data and `String.mapi`:

In [ ]:
let toCamelCase word indexes =
    let mapping i c =
        match (indexes |> List.contains i) with
        | true                      -> Char.ToUpper(c)
        | _ when Char.IsUpper(c)    -> Char.ToLower(c)
        | _                         -> c

    word |> String.mapi mapping

[
    ("fsharP", [0; 1])
    ("nAtiveinterop", [0; 6])
    ("taskbuildereXtensions", [0; 4; 11])
    ("microsoftword", [0; 9])
]
|> List.map (fun (word, indexes) -> (word, indexes) ||> toCamelCase)

index,value
0,FSharp
1,NativeInterop
2,TaskBuilderExtensions
3,MicrosoftWord


## let `List.map` drive function design

The above use of `fun` is too verbose:

```fsharp
List.map (fun (word, indexes) -> (word, indexes) ||> toCamelCase)
```

After asking [a StackOverflow question](https://stackoverflow.com/questions/70388340/do-fun-lambda-expressions-have-shorthand-syntax), the verbosity is coming from the curry form `toCamelCase` which is not compatible with the tuple form of `List.map`:

In [ ]:
let toCamelCase (word, indexes) =
    let mapping i c =
        match (indexes |> List.contains i) with
        | true                      -> Char.ToUpper(c)
        | _ when Char.IsUpper(c)    -> Char.ToLower(c)
        | _                         -> c

    word |> String.mapi mapping

[
    ("fsharP", [0; 1])
    ("nAtiveinterop", [0; 6])
    ("taskbuildereXtensions", [0; 4; 11])
    ("microsoftword", [0; 9])
]
|> List.map toCamelCase

index,value
0,FSharp
1,NativeInterop
2,TaskBuilderExtensions
3,MicrosoftWord


When it is _not_ possible to change `toCamelCase` then the issue of ‘un-currying’ suddenly becomes important.

In 2018, [Mark Seemann showed us](https://blog.ploeh.dk/2018/02/05/uncurry-isomorphisms/) that F# has had no built-in `curry` or `uncurry` functions (which Haskell has) but they are easy to make:

```fsharp
let curry f x y = f (x, y)
let uncurry f (x, y) = f x y
```

## implied un-currying?

The use of the double pipeline operator, `||>`, in the example above is effectively _decomposing_ the tuple for a curried function:

```fsharp
(word, indexes) ||>
```

This decomposition is effectively _un-currying_—which implies that what was regarded as verbosity is actually expressing un-currying:

```fsharp
List.map (fun (word, indexes) -> (word, indexes) ||> toCamelCase)
```

@[BryanWilhite](https://twitter.com/BryanWilhite)
